In [11]:
# Part 1
import re
import itertools

data = open("input_12_test.txt").read().splitlines()
data = [x.split() for x in data]
[print(line) for line in data]

total_combos = 0
# for each line in data:
print(data)
line = data[0]
cond = line[0]
for i,line in enumerate(data):
    key = [int(x) for x in line[1].split(',')]
    # Set up the line[0]ing for searching
    line[0] = line[0].join(['.','.']) # add '.' at the ends to help search line[0]ings later
    unknown = [i for i, letter in enumerate(line[0]) if letter == '?']
    broken = [x.start(0) for x in re.finditer(r'#',line[0])]
    print(broken,unknown)
    print(line[0],'key =',key,'broken =',broken,'unknown =',unknown)
    test_combos = list(itertools.combinations(unknown, sum(key)-len(broken)))
    print(test_combos)
    line[0] = re.sub(r'\?',r'.',line[0])  # fill in the ? with .
    print(line[0])
    print(' ')

    # loop over the possible combinations of broken springs and validate
    for k,combo in enumerate(test_combos):
        temp = line[0]
        # temp[combo] = ''
        temp = [ch for ch in temp]
        print(temp)
        for m in range(len(combo)):
            temp[combo[m]] = '#'
        temp = ''.join(temp)
        print(temp)
        1





total_combos = 0
    # while not tried_all:
    #     1




['???.###', '1,1,3']
['.??..??...?##.', '1,1,3']
['?#?#?#?#?#?#?#?', '1,3,1,6']
['????.#...#...', '4,1,1']
['????.######..#####.', '1,6,5']
['?###????????', '3,2,1']
[['???.###', '1,1,3'], ['.??..??...?##.', '1,1,3'], ['?#?#?#?#?#?#?#?', '1,3,1,6'], ['????.#...#...', '4,1,1'], ['????.######..#####.', '1,6,5'], ['?###????????', '3,2,1']]
[5, 6, 7] [1, 2, 3]
.???.###. key = [1, 1, 3] broken = [5, 6, 7] unknown = [1, 2, 3]
[(1, 2), (1, 3), (2, 3)]
.....###.
 
['.', '.', '.', '.', '.', '#', '#', '#', '.']
['.', '#', '#', '.', '.', '#', '#', '#', '.']
.##..###.
['.', '.', '.', '.', '.', '#', '#', '#', '.']
['.', '#', '.', '#', '.', '#', '#', '#', '.']
.#.#.###.
['.', '.', '.', '.', '.', '#', '#', '#', '.']
['.', '.', '#', '#', '.', '#', '#', '#', '.']
..##.###.
[12, 13] [2, 3, 6, 7, 11]
..??..??...?##.. key = [1, 1, 3] broken = [12, 13] unknown = [2, 3, 6, 7, 11]
[(2, 3, 6), (2, 3, 7), (2, 3, 11), (2, 6, 7), (2, 6, 11), (2, 7, 11), (3, 6, 7), (3, 6, 11), (3, 7, 11), (6, 7, 11)]
............

In [6]:
temp = line[0]
print(temp)
combo = test_combos[0]
temp = [ch for ch in temp]
print(temp)
for k in range(len(combo)):
    temp[combo[k]] = combo[k]
temp = ''.join([temp])
print(temp)


..###.........
['.', '.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.']


TypeError: sequence item 0: expected str instance, list found